In [1]:
#VARIABLES to set up
# SETUP the name of the file where result will be write
SF = 10 #choose between 1,3,5,8,10
maintenance_table_type1= [ 
                            #"catalog_sales", 
                            "catalog_returns",
                          "inventory","store_returns",
                          #"store_sales",
                            "web_returns",
                          #"web_sales"
]

In [2]:
#import
import os
import time
import datetime
from pyspark.sql import SparkSession
import glob

# Initialize SparkSession
spark = SparkSession.builder.appName("TPC-DS Data Loading").config("spark.sql.catalogImplementation", "hive").getOrCreate()

23/10/28 13:54:59 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.178.59 instead (on interface eno1)
23/10/28 13:54:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/28 13:55:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#create all tables
with open(os.getcwd() + "/ddl/Create_database.sql", 'r') as file: # path/to/tpcds.sql
    db_string = file.read()

db_string = str.split(db_string, ";")
db_string = db_string[:-1]

for string in db_string:
    if string != '\n' or string != '':
        spark.sql(string)

23/10/28 13:55:04 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/28 13:55:04 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/10/28 13:55:26 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/10/28 13:55:26 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore coucou@127.0.1.1
23/10/28 13:55:26 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
23/10/28 13:55:29 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/10/28 13:55:29 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAu

23/10/28 13:55:32 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/10/28 13:55:32 WARN HiveMetaStore: Location: file:/home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DS/spark-warehouse/warehouse specified for non-external table:warehouse
23/10/28 13:55:32 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/10/28 13:55:32 WARN HiveMetaStore: Location: file:/home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DS/spark-warehouse/web_page specified for non-external table:web_page
23/10/28 13:55:33 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefaul

In [4]:
#load datas into tables
table_loading =[]
data_file = glob.glob(os.getcwd() + "/data/data"+ str(SF)+"/*")
for i in data_file:
    table_name = i.split("/")[-1].split(".")[0]
    if table_name != "dbgen_version":
        sql_command = "select * from " + table_name
        tmp=spark.sql(sql_command)
        table_load_start = time.time()
        df = spark.read.schema(tmp.schema).csv( i, sep='|')
        df.write.mode("append").insertInto(table_name)
        table_load_end = time.time()
        table_loading.append([table_name, table_load_end - table_load_start])
        print("Data from table " + table_name + " is inserted")
        
        
#write result        
f = open("result/table_time_SF"+ str(SF) + ".csv", "w")
f.write( str(table_loading) +'\n')
f.close()

23/10/28 13:55:43 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


Data from table customer is inserted


Data from table customer_address is inserted
Data from table promotion is inserted


23/10/28 13:55:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Data from table date_dim is inserted
Data from table ship_mode is inserted
Data from table store is inserted


...

Data from table customer_demographics is inserted


Data from table store_returns is inserted


Data from table web_sales is inserted
Data from table income_band is inserted


Data from table catalog_returns is inserted


Data from table inventory is inserted


In [5]:
#run queries
queries_files = glob.glob(os.getcwd() + "/Queries/query*")
time1=[0 for i in range(99)]
time2=[0 for i in range(99)]
time3=[0 for i in range(99)]
time4=[0 for i in range(99)]
time5=[0 for i in range(99)]

for i in queries_files:
    with open( i , 'r') as file:
        db_string = file.read()
    db_string=db_string.split(";")[:-1]
    #run same query 5 times
    number_queries = int(i.split("query")[-1].split(".")[0])
    for i in db_string:
        exec_start = time.time()
        df = spark.sql(i)
        exec_end = time.time()
        time1[number_queries-1] += exec_end- exec_start
        exec_start = time.time()
        spark.sql(i)
        exec_end = time.time()
        time2[number_queries-1] += exec_end- exec_start
        exec_start = time.time()
        spark.sql(i)
        exec_end = time.time()
        time3[number_queries-1] += exec_end- exec_start
        exec_start = time.time()
        spark.sql(i)
        exec_end = time.time()
        time4[number_queries-1] += exec_end- exec_start
        exec_start = time.time()
        spark.sql(i)
        exec_end = time.time()
        time5[number_queries-1] += exec_end- exec_start


    
    #end time
f = open("result/sf"+ str(SF) + ".csv", "w")
f.write(str(time1)+ '\n' + str(time2)+ '\n'+ str(time3)+ '\n'+ str(time4)+ '\n' + str(time5))
f.close()

In [6]:
#Throughput TEST 1
time_trought=[0,0,0,0,0,0,0,0]
time_maintenance = [0,0]
for k in range(2): #boucle on Throughput TEST
    startStreamNumber = str(k*4+1)
    endStreamNumber = str(k*4+4)
    for i in range(k*4+1,k*4+5): #boucle on stream
        queries_files = glob.glob(os.getcwd() + "/Query_"+ startStreamNumber +"-"+ endStreamNumber +"/query*_stream"+ str(i)+"*")
        for j in queries_files:
            with open( j , 'r') as file:
                db_string = file.read()


            db_string=db_string.split(";")[:-1]
            for m in db_string:
                exec_start = time.time()
                spark.sql(m)
                exec_end = time.time()
                time_trought[i-1] += exec_end- exec_start
    
    #maintenance of type 1 inserting
    maintenance_start = time.time()
    for i in maintenance_table_type1: #name of table
        sql_command = "select * from " + str(i)
        tmp=spark.sql(sql_command)
        for j in range(k*4+1,k*4+5):
            df = spark.read.schema(tmp.schema).csv(os.getcwd() + "/maintenance/maintenance"+ str(SF)+"/s_"+ str(i) +"_"+str(j)+".dat", sep='|')
            #df.write.mode("append").insertInto(str(i))
    
    
    
    # methods of type 2#
    for j in range(k*4+1,k*4+5):
        with open(os.getcwd() + "/maintenance/maintenance"+ str(SF)+ "/delete_"+ str(j)+".dat", 'r') as file:
            db_string = file.read()
        db_string = db_string.split("|")[:-1]
        for l in range(0,len(db_string),2):
            method_2_string = """
                --> DF_CS <--
                DELETE cr
                FROM catalog_returns AS cr
                JOIN catalog_sales AS cs ON cr.cr_order_number = cs.cs_order_number AND cr.cr_item_sk = cs.cs_item_sk
                JOIN date_dim AS dd ON cs.cs_sold_date_sk = dd.d_date_sk
                WHERE dd.d_date BETWEEN """ + db_string[l] + """ AND """ +db_string[l+1]+""";

                DELETE cs
                FROM catalog_sales AS cs
                JOIN date_dim AS dd ON cs.cs_sold_date_sk = dd.d_date_sk
                WHERE dd.d_date BETWEEN """ + db_string[l] + """ AND """ +db_string[l+1]+""";

                --> DF_SS <--
                DELETE sr
                FROM store_returns AS sr
                JOIN store_sales AS ss ON sr.sr_ticket_number = ss.ss_ticket_number AND sr.sr_item_sk = ss.ss_item_sk
                JOIN date_dim AS dd ON ss.ss_sold_date_sk = dd.d_date_sk
                WHERE dd.d_date BETWEEN """ + db_string[l] + """ AND """ +db_string[l+1]+""";

                DELETE ss
                FROM store_sales AS ss
                JOIN date_dim AS dd ON ss.ss_sold_date_sk = dd.d_date_sk
                WHERE dd.d_date BETWEEN """ + db_string[l] + """ AND """ +db_string[l+1]+""";

                --> DF_WS <--
                DELETE wr
                FROM web_returns AS wr
                JOIN web_sales AS ws ON wr.wr_order_number = ws.ws_order_number AND wr.wr_item_sk = ws.ws_item_sk
                JOIN date_dim AS dd ON ws.ws_sold_date_sk = dd.d_date_sk
                WHERE dd.d_date BETWEEN """ + db_string[l] + """ AND """ +db_string[l+1]+""";

                DELETE ws
                FROM web_sales AS ws
                JOIN date_dim AS dd ON ws.ws_sold_date_sk = dd.d_date_sk
                WHERE dd.d_date BETWEEN """ + db_string[l] + """ AND """ +db_string[l+1]+""";

                """
            method_2_string = method_2_string.split(";")[:-1]
            for sql_command in method_2_string:
                #spark.sql(sql_command)
                pass

    
    #maintenance of type 3 # delete inventory
    for j in range(k*4+1,k*4+5):
        with open(os.getcwd() + "/maintenance/maintenance"+ str(SF)+ "/inventory_delete_"+ str(j)+".dat", 'r') as file:
            db_string = file.read()
        db_string = db_string.split("|")[:-1]
        for l in range(0,len(db_string),2):
            sql_command = """DELETE FROM inventory i JOIN date_dim d ON i.inv_date_sk = d.d_date_sk WHERE d.d_date BETWEEN""" + db_string[l] + """ AND """ +db_string[l+1]
            #spark.sql(sql_command)
    maintenance_end = time.time()
    time_maintenance[k] = maintenance_end - maintenance_start
f = open("result/ThroughTest_and_Maintenance_sf"+ str(SF) + ".csv", "w")
f.write(str(time_trought)+'\n' + str(time_maintenance))
f.close()